# Data Science Final Project - EE 379k - sp18 - 

# Shahshank Kambhampati - skk834, Shrikara Murthy - svm456, Pranav Harathi - , Neil Charles - 

# Job Satisfaction Prediction

## Imports

### All hail lord and savior XGB

In [56]:
import numpy as np
import pandas as pd
import re

import xgboost as xgb
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d
from scipy import stats
from sklearn.linear_model import RidgeCV, LassoCV, LogisticRegression, Ridge, Lasso
from sklearn.cross_decomposition import PLSRegression
from sklearn.ensemble import RandomTreesEmbedding, AdaBoostClassifier, AdaBoostRegressor, RandomForestRegressor, RandomForestClassifier, VotingClassifier
from sklearn.metrics import make_scorer, roc_auc_score, accuracy_score, mean_squared_error
from sklearn.model_selection import LeaveOneOut, train_test_split, GridSearchCV, cross_val_score
from sklearn.feature_selection import RFE,RFECV
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn import metrics
from sklearn.ensemble import ExtraTreesClassifier, ExtraTreesRegressor
from sklearn.feature_selection import SelectKBest,  chi2
from sklearn.svm import SVR

%config inlinebackend.figure_format = 'retina' 
%matplotlib inline

# Initial Observations about the Data and Pre-Processing

In [2]:
number = re.compile('[\d,]+')
def get_first_number(val):
    matched = number.match(str(val))
    if matched:
        return int(matched.group())
    else:
        return np.nan

satisfaction_strs = {
    'I love my job': 10,
    'I\'m somewhat satisfied with my job': 7.5,
    'I\'m neither satisfied nor dissatisfied with my job': 5,
    'I\'m somewhat dissatisfied with my job': 2.5,
    'I hate my job': 0,
}

binary_labels = [
    'Lang & Tech',
    'Training & Education',
    'How can companies improve interview process',
    'Why try Stack Overflow Careers',
    'Most important aspect of new job opportunity',
    'Most annoying about job search',
    'Appealing message traits',
    'Most urgent info about job opportunity',
    'Who do you want to communicate with about a new job opportunity',
    'Why use Stack Overflow',
    'Why answer',
    'Source control used',
]

numeric_labels = [
    'Age',
    'Years IT / Programming Experience',
    'Compensation: midpoint'
]

yes_no_labels = [
    'Changed Jobs in last 12 Months'
]


In [3]:
# 2015 preproc
data = pd.read_csv('data/2015.csv')
to_drop = [label for label in data if 'write-in' in label.lower()]

data['Job Satisfaction'] = data['Job Satisfaction']\
                                    .map(satisfaction_strs).astype('float')
data = data[data['Job Satisfaction'].notnull()]
to_drop.append('Country')
to_drop.append('Compensation')

data['Age'] = data['Age'].map(get_first_number).astype('float')
data['gender_M'] = (data['Gender'] == 'Male').astype('int8')
data['gender_F'] = (data['Gender'] == 'Female').astype('int8')
to_drop.append('Gender')
to_drop.append('Prefered Source Control')

bin_labels = [key for key in data if any(label in key for label in binary_labels)]
data[bin_labels] = data[bin_labels].apply(lambda col: col.notnull().astype('int8'))

data[numeric_labels] = data[numeric_labels].applymap(get_first_number)
data[yes_no_labels] = data[yes_no_labels]\
                                .apply(lambda col: col.map({'Yes': 1, 'No': 0}))\
                                .fillna(0)

data.drop(to_drop, axis=1, inplace=True)
data = pd.get_dummies(data)

print 'Columns with NaNs:'
for key in data:
    if data[key].isnull().any():
        print key
data.fillna(data.mean(), inplace=True)
print 'Filling with mean of column'

/home/shrikar/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (5,108,121,196,197,198) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Columns with NaNs:
Age
Years IT / Programming Experience
Compensation: midpoint
Filling with mean of column


In [7]:
# data contains a DataFrame with no NaNs, all numbers.
# We're trying to predict the "Job Satisfaction" column.

In [8]:
X = data.drop(['Job Satisfaction'], axis=1)
y = data['Job Satisfaction']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,stratify=y)#stratify=y
scoring = {'mean': make_scorer(mean_squared_error)}

In [9]:
print data.head()

    Age  Years IT / Programming Experience  Current Lang & Tech: Android  \
0  25.0                                2.0                             0   
1  20.0                                1.0                             0   
2  20.0                                1.0                             0   
3  25.0                                6.0                             0   
4  30.0                                2.0                             0   

   Current Lang & Tech: Arduino  Current Lang & Tech: AngularJS  \
0                             0                               0   
1                             0                               0   
2                             0                               0   
3                             0                               0   
4                             0                               0   

   Current Lang & Tech: C  Current Lang & Tech: C++  \
0                       0                         0   
1                       0     

In [10]:
print data.describe()

                Age  Years IT / Programming Experience  \
count  16004.000000                       16004.000000   
mean      28.478348                           5.839325   
std        6.955481                           3.850501   
min       20.000000                           1.000000   
25%       25.000000                           2.000000   
50%       25.000000                           6.000000   
75%       30.000000                          11.000000   
max       51.000000                          11.000000   

       Current Lang & Tech: Android  Current Lang & Tech: Arduino  \
count                  16004.000000                  16004.000000   
mean                       0.173269                      0.061110   
std                        0.378492                      0.239539   
min                        0.000000                      0.000000   
25%                        0.000000                      0.000000   
50%                        0.000000                      0.0000

In [11]:
for i in data.corr():
    print i

Age
Years IT / Programming Experience
Current Lang & Tech: Android
Current Lang & Tech: Arduino
Current Lang & Tech: AngularJS
Current Lang & Tech: C
Current Lang & Tech: C++
Current Lang & Tech: C++11
Current Lang & Tech: C#
Current Lang & Tech: Cassandra
Current Lang & Tech: CoffeeScript
Current Lang & Tech: Cordova
Current Lang & Tech: Clojure
Current Lang & Tech: Cloud
Current Lang & Tech: Dart
Current Lang & Tech: F#
Current Lang & Tech: Go
Current Lang & Tech: Hadoop
Current Lang & Tech: Haskell
Current Lang & Tech: iOS
Current Lang & Tech: Java
Current Lang & Tech: JavaScript
Current Lang & Tech: LAMP
Current Lang & Tech: Matlab
Current Lang & Tech: MongoDB
Current Lang & Tech: Node.js
Current Lang & Tech: Objective-C
Current Lang & Tech: Perl
Current Lang & Tech: PHP
Current Lang & Tech: Python
Current Lang & Tech: R
Current Lang & Tech: Redis
Current Lang & Tech: Ruby
Current Lang & Tech: Rust
Current Lang & Tech: Salesforce
Current Lang & Tech: Scala
Current Lang & Tech: Shar

In [13]:
model = xgb.XGBRegressor()
rfe = RFE(model, 50,2)
rfe = rfe.fit(X_train,y_train)
X_train = rfe.transform(X_train)
X_test = rfe.transform(X_test)
#rfe_preds = rfe.predict(X_test)
#print mean_squared_error(y_test,rfe_preds)


ValueError: X has a different shape than during fitting.

In [ ]:
parameters = {
    'n_estimators':[160]
}
modelETC = ExtraTreesRegressor()
gridETC = GridSearchCV(modelETC, param_grid=parameters,scoring = scoring, cv=5, refit = 'mean',verbose=1)
gridETC.fit(X_train,y_train)
print(gridETC.best_params_)
#print(modelETC.feature_importances_)
etc_preds = gridETC.predict(X_test)
print mean_squared_error(y_test,etc_preds)

In [ ]:
print mean_squared_error(y_test,etc_preds)

In [35]:
parameters = { #when use hyperthread, xgboost may become slower
              #'objective':['binary:logistic'],
             # 'booster':['dart'],
               'booster':['gbtree'],
              'learning_rate': [.1], #so called `eta` value
              'max_depth': [9],#
              'min_child_weight': [5],#
              'reg_alpha':[.1],
              'silent': [0],
              'subsample': [.7],
              'gamma':[.3],
              'colsample_bytree': [.8],
              'n_estimators': [1000], #number of trees, change it to 1000 for better results
              'seed': [1337]
}

In [57]:
xgb_model = xgb.XGBRegressor()
#gridXGB = GridSearchCV(xgb_model, param_grid=parameters,scoring = scoring, cv=5, verbose=2, refit = 'mean')
#gridXGB.fit(X_train, y_train,eval_metric='rmse')
#print(gridXGB.best_params_)
#xgb_preds = gridXGB.predict(X_test)
xgb_model.fit(X_train, y_train)
xgb_preds = xgb_model.predict(X_test)
print mean_squared_error(y_test,xgb_preds)**.5


2.2682161898


In [51]:
print sum(y_test)/len(y_test)
print sum(xgb_preds)/len(xgb_preds)

7.5
7.50948945377


In [52]:
for pred, clipped, actual in zip(xgb_preds, clipS(xgb_copy), y_test ):
    print pred, clipped, actual

8.7683 7.5 10.0
6.2244 7.5 10.0
6.93961 7.5 7.5
7.54146 7.5 10.0
7.46333 7.5 7.5
8.51323 7.5 10.0
9.27608 10 10.0
8.91444 10 5.0
6.76045 7.5 7.5
8.93987 10 10.0
8.59289 7.5 10.0
7.13907 7.5 10.0
8.09517 7.5 10.0
8.90442 10 7.5
7.60538 7.5 7.5
7.85177 7.5 10.0
6.71438 7.5 7.5
8.9704 10 10.0
7.87946 7.5 7.5
7.68008 7.5 7.5
7.38076 7.5 7.5
6.16779 7.5 7.5
6.82522 7.5 2.5
7.52401 7.5 7.5
6.16789 7.5 5.0
7.42272 7.5 7.5
8.40539 7.5 10.0
6.14735 7.5 5.0
7.7019 7.5 7.5
8.05244 7.5 7.5
8.57009 7.5 7.5
6.90616 7.5 7.5
8.94493 10 7.5
7.66372 7.5 10.0
6.8824 7.5 7.5
6.87881 7.5 7.5
8.96877 10 10.0
8.15966 7.5 7.5
4.62106 5 2.5
8.32668 7.5 10.0
7.22841 7.5 7.5
9.77668 10 10.0
7.05223 7.5 7.5
7.46531 7.5 0.0
6.96507 7.5 7.5
7.42393 7.5 5.0
7.55165 7.5 5.0
6.55768 7.5 7.5
6.71437 7.5 7.5
7.11866 7.5 7.5
7.40455 7.5 7.5
6.97156 7.5 5.0
7.51808 7.5 7.5
9.41673 10 10.0
7.72523 7.5 5.0
6.85657 7.5 7.5
8.74519 7.5 10.0
7.62139 7.5 7.5
4.74984 5 10.0
8.54628 7.5 10.0
6.78358 7.5 2.5
7.59076 7.5 5.0
7.3913

7.09444 7.5 10.0
7.52268 7.5 5.0
7.77376 7.5 10.0
7.25529 7.5 10.0
3.90724 5 2.5
6.46163 7.5 7.5
9.18204 10 10.0
6.91872 7.5 10.0
7.36436 7.5 7.5
4.28752 5 5.0
8.96765 10 7.5
6.98294 7.5 7.5
6.98453 7.5 5.0
7.68175 7.5 10.0
9.3011 10 10.0
7.65063 7.5 7.5
8.4545 7.5 10.0
7.19469 7.5 10.0
7.12244 7.5 10.0
6.42847 7.5 7.5
8.17812 7.5 7.5
8.16843 7.5 7.5
7.23971 7.5 7.5
9.16684 10 10.0
7.31308 7.5 10.0
6.64815 7.5 10.0
8.8834 7.5 10.0
9.26225 10 10.0
5.8112 5 5.0
7.28885 7.5 5.0
8.57435 7.5 10.0
8.30811 7.5 7.5
8.93753 10 7.5
8.33812 7.5 7.5
9.10841 10 7.5
7.159 7.5 10.0
7.70017 7.5 10.0
8.05879 7.5 7.5
6.50428 7.5 7.5
7.63072 7.5 7.5
9.39451 10 10.0
8.29039 7.5 7.5
8.05648 7.5 10.0
9.14314 10 7.5
7.09552 7.5 10.0
7.69711 7.5 10.0
7.44506 7.5 5.0
7.65885 7.5 7.5
7.14298 7.5 5.0
8.66531 7.5 7.5
7.67469 7.5 5.0
9.01425 10 10.0
9.03049 10 10.0
5.40697 5 7.5
9.15863 10 10.0
9.61584 10 10.0
9.33748 10 10.0
7.51078 7.5 10.0
9.22063 10 7.5
5.20851 5 7.5
8.30684 7.5 10.0
9.46122 10 10.0
5.23103 5 

7.61804 7.5 10.0
6.7986 7.5 7.5
6.17554 7.5 7.5
6.98009 7.5 7.5
8.09638 7.5 10.0
10.1248 10 10.0
4.86739 5 10.0
7.14238 7.5 7.5
6.69431 7.5 7.5
9.02457 10 10.0
8.63699 7.5 10.0
8.65008 7.5 7.5
5.5052 5 10.0
9.31934 10 10.0
5.78726 5 5.0
6.17972 7.5 10.0
7.26604 7.5 7.5
7.80111 7.5 7.5
5.37685 5 2.5
8.78195 7.5 10.0
6.42375 7.5 7.5
8.47546 7.5 10.0
8.95023 10 10.0
7.3111 7.5 10.0
7.42979 7.5 2.5
6.85539 7.5 10.0
7.08396 7.5 7.5
9.21312 10 10.0
8.32344 7.5 10.0
7.63836 7.5 10.0
9.44599 10 10.0
8.39627 7.5 10.0
7.08264 7.5 5.0
4.45791 5 2.5
8.84931 7.5 10.0
8.44016 7.5 7.5
6.73216 7.5 7.5
7.20387 7.5 5.0
5.36718 5 5.0
7.69096 7.5 7.5
7.66901 7.5 10.0
8.45237 7.5 7.5
6.8107 7.5 7.5
7.54776 7.5 10.0
6.82055 7.5 7.5
7.11465 7.5 7.5
5.59456 5 2.5
7.56626 7.5 10.0
7.23957 7.5 7.5
8.26615 7.5 10.0
9.16458 10 10.0
6.65812 7.5 7.5
6.27907 7.5 2.5
7.13956 7.5 7.5
7.13249 7.5 2.5
7.06605 7.5 7.5
7.5553 7.5 7.5
8.84067 7.5 7.5
6.82759 7.5 2.5
7.37105 7.5 10.0
7.01255 7.5 10.0
7.65431 7.5 7.5
6.96314

9.64307 10 10.0
7.13565 7.5 5.0
6.83832 7.5 10.0
7.10222 7.5 5.0
8.30778 7.5 10.0
6.59424 7.5 0.0
5.76181 5 7.5
6.33346 7.5 2.5
8.40543 7.5 10.0
6.70003 7.5 7.5
8.41152 7.5 10.0
7.1919 7.5 5.0
6.97794 7.5 10.0
8.53562 7.5 7.5
6.98169 7.5 7.5
7.24531 7.5 2.5
8.50342 7.5 10.0
7.29237 7.5 2.5
7.92472 7.5 7.5
6.75165 7.5 7.5
6.91898 7.5 7.5
7.38857 7.5 5.0
8.86905 7.5 10.0
8.58755 7.5 7.5
8.68959 7.5 10.0
6.8776 7.5 7.5
7.13071 7.5 7.5
7.38245 7.5 7.5
7.42395 7.5 7.5
9.38946 10 10.0
7.15466 7.5 10.0
4.68606 5 0.0
6.80861 7.5 10.0
6.72928 7.5 7.5
8.71721 7.5 10.0
8.51392 7.5 10.0
7.3065 7.5 7.5
6.78781 7.5 7.5
9.84138 10 10.0
7.98957 7.5 10.0
7.71603 7.5 10.0
7.48932 7.5 10.0
7.53809 7.5 7.5
7.42943 7.5 10.0
8.13203 7.5 7.5
8.46125 7.5 5.0
7.84926 7.5 10.0
9.32189 10 7.5
6.69401 7.5 7.5
7.75254 7.5 7.5
3.92068 5 2.5
8.70602 7.5 2.5
9.0322 10 10.0
6.83797 7.5 7.5
6.9785 7.5 7.5
7.13582 7.5 10.0
8.49983 7.5 10.0
7.86491 7.5 10.0
7.58028 7.5 5.0
7.05827 7.5 7.5
8.89942 7.5 10.0
7.39665 7.5 7.5

9.88281 10 7.5
6.9752 7.5 7.5
7.25934 7.5 2.5
6.39731 7.5 2.5
6.86957 7.5 2.5
9.40651 10 10.0
6.66716 7.5 7.5
7.30932 7.5 7.5
6.76767 7.5 10.0
8.57621 7.5 10.0
6.44798 7.5 10.0
7.379 7.5 7.5
7.79962 7.5 10.0
4.98502 5 2.5
6.89313 7.5 2.5
8.16216 7.5 2.5
6.32479 7.5 7.5
7.84987 7.5 7.5
9.20412 10 10.0
7.85815 7.5 10.0
7.14435 7.5 7.5
5.25315 5 2.5
5.93004 5 5.0
6.94727 7.5 10.0
6.38786 7.5 7.5
7.03315 7.5 7.5
8.83403 7.5 10.0
6.11893 7.5 0.0
8.98245 10 10.0
7.48159 7.5 10.0
7.21375 7.5 10.0
7.11343 7.5 10.0
7.15505 7.5 10.0
9.11234 10 10.0
8.34808 7.5 7.5
9.79472 10 10.0
6.41044 7.5 7.5
6.71345 7.5 7.5
7.77146 7.5 10.0
5.65524 5 5.0
6.76576 7.5 2.5
7.26188 7.5 7.5
7.1784 7.5 7.5
4.29649 5 2.5
9.24551 10 10.0
8.22509 7.5 10.0
6.76308 7.5 10.0
4.61548 5 5.0
9.23354 10 10.0
7.32463 7.5 7.5
8.37201 7.5 7.5
7.38676 7.5 7.5
9.31665 10 10.0
8.4262 7.5 10.0
6.61135 7.5 10.0
6.7392 7.5 5.0
7.00279 7.5 7.5
7.29189 7.5 2.5
6.68963 7.5 7.5
7.39325 7.5 10.0
7.24645 7.5 0.0
7.24675 7.5 5.0
8.57558 7.

In [49]:
def clipS(preds):
    for pred, element in enumerate(preds):
        if preds[pred] < 1.25:
            preds[pred] = 0
        elif preds[pred] < 3.75:
            preds[pred] = 2.5
        elif  preds[pred] < 6.25:
            preds[pred] = 5
        elif  preds[pred] < 8.75:
            preds[pred] = 7.5
        else: 
            preds[pred] = 10
    return preds

In [58]:
print mean_squared_error(y_test,clipS(xgb_preds))**.5

2.37806554825
